<a href="https://colab.research.google.com/github/MKJJang/airflow/blob/master/practice_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)
# input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']


In [3]:
str2idx = {word:idx for idx, word  in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

# str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
# idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
# input_ids:  [0, 1, 2, 3, 4]


str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


In [4]:
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

In [5]:
import torch
input_embeddings = embed_layer(torch.tensor(input_ids))
input_embeddings = input_embeddings.unsqueeze(0)
input_embeddings.shape
input_embeddings = input_embeddings.unsqueeze(0)
input_embeddings.shape

# 1개의 문장이고 5개의 토큰이 있고 16차원의 임베딩이 생성되었음을 확인할 수 있다.
# torch.Size([1, 1, 5, 16])

torch.Size([1, 1, 5, 16])

In [6]:
embedded_dim = 16
max_position = 12
embeded_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
toked_embeddings = embed_layer(torch.tensor(input_ids)) # (5,16)
token_embeddings = toked_embeddings.unsqueeze(0) # (1,5,16)
token_embeddings = toked_embeddings + position_encodings
input_embeddings.shape

# torch.Size([1, 1, 5, 16])

torch.Size([1, 1, 5, 16])

In [9]:
# 예제 2.4. 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

head_dim = 16

#쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
querys = weight_q(token_embeddings) #(1,5,6)
keys = weight_k(token_embeddings) #(1,5,6)
values = weight_v(token_embeddings) #(1,5,6)

print(querys)
print(keys)
print(values)

tensor([[[-0.6162,  1.0827, -0.2643,  1.6744, -0.0235, -0.7401,  0.5347,
          -0.2917, -0.1730, -1.6108, -0.4645,  0.4613,  0.6690,  0.6273,
          -0.3162, -0.0382],
         [-0.6820,  0.8528,  0.2134,  0.2797, -0.5687, -1.0554,  0.1513,
          -1.6495,  0.2305, -2.6452, -0.0979,  1.2764, -0.0178,  0.5187,
          -0.3336,  0.7411],
         [ 0.4693, -1.3521, -1.0545, -0.2039,  0.6332, -0.2554,  0.1118,
           0.1425, -0.2897, -0.0209,  0.0338,  0.4834,  0.1068,  0.0790,
           0.1575, -0.7535],
         [-0.9703,  0.9017, -0.5385,  1.2906,  1.1570, -0.2884, -1.4554,
           0.6264,  1.0711, -2.2912, -1.1444,  0.7135,  0.1785,  1.0821,
           0.4626, -0.3524],
         [ 0.8193,  0.1722, -1.1576, -0.1091, -1.7512, -0.6975,  0.5769,
           1.1210,  0.7730,  0.2605,  0.0962, -1.5100,  0.1112, -0.9559,
           0.2537, -1.0503]]], grad_fn=<ViewBackward0>)
tensor([[[-1.2399e+00, -1.9895e+00,  8.9470e-02, -1.3158e-01,  6.8075e-01,
          -3.8095e-01, 

In [26]:
# 예제 2.5 스케일 점곱 방식의 어텐션

from math import sqrt
import torch.nn.functional as F
def compute_attention(querys, keys, values, is_causal=False):
  dim_k = querys.size(-1) #16
  scores = querys @ keys.transpose(-2,-1) /sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return weights @ values

In [27]:
# 예제 2.6 어텐션 연산의 입력과 출력
print("원본 입력 형태: ", input_embeddings.shape)
import torch
import transformers
after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)

# 원본 입력 형태:  torch.Size([1, 1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])

원본 입력 형태:  torch.Size([1, 1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


In [34]:
#예제 2.7 어텐션 연산을 수행하는 AttentionHead 클레스

class AttentionHead(nn.Module):
  def __init__(self, token_embeded_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embeded_dim, head_dim) #  쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embeded_dim, head_dim) #  키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embeded_dim, head_dim) #  값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),
        self.weight_k(keys),
        self.weight_v(values),
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedded_dim, embedded_dim)
after_attention_embbedings = attention_head(input_embeddings, input_embeddings, input_embeddings)

In [37]:
print(attention_head)

AttentionHead(
  (weight_q): Linear(in_features=16, out_features=16, bias=True)
  (weight_k): Linear(in_features=16, out_features=16, bias=True)
  (weight_v): Linear(in_features=16, out_features=16, bias=True)
)


In [36]:
print(after_attention_embbedings)

tensor([[[[-0.3197,  0.3011,  0.1351, -0.0755,  0.0653,  0.2729, -0.2246,
            0.3369,  0.0901, -0.1661, -0.3104,  0.0417, -0.0414, -0.3447,
            0.2401,  0.1230],
          [-0.3747,  0.3352,  0.1125, -0.0608,  0.0503,  0.2187, -0.2044,
            0.2766, -0.0044, -0.1856, -0.3022,  0.0147, -0.0613, -0.3669,
            0.2010,  0.1616],
          [-0.2649,  0.3924,  0.1248, -0.1189,  0.0384,  0.2022, -0.1723,
            0.2180, -0.0122, -0.0596, -0.3699,  0.1277, -0.0149, -0.2565,
            0.1836,  0.1869],
          [-0.2916,  0.2133,  0.1664, -0.1265,  0.0589,  0.3189, -0.2263,
            0.4053,  0.1717, -0.1647, -0.3186, -0.0025, -0.0381, -0.3235,
            0.2861,  0.0466],
          [-0.4935,  0.3335,  0.0784, -0.0309,  0.0688,  0.2022, -0.2163,
            0.2436, -0.0340, -0.2554, -0.2800, -0.0092, -0.0835, -0.4443,
            0.1949,  0.1787]]]], grad_fn=<UnsafeViewBackward0>)


In [38]:
#2.8 멀티 헤드 어텐션 구현

class MultiheadAttention(nn.Module):
  def __init(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear= nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    B, T, C = querys.size()
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
    attentions = compute_attention(querys, keys, values, is_causal=self.is_causal)
    attentions = attentions.transpose(1,2).contiguous().view(B, T, C)
    outputs = self.concat_linear(attentions)
    return outputs

  n_head = 4
  multihead_attention = MultiheadAttention(embedded_dim, embedded_dim, n_head)
  after_attention_embeddings = multihead_attention(input_embeddings, input_embeddings, input_embeddings)
  print(after_attention_embeddings)

